In [1]:
!pip install litellm

In [2]:
from IPython.display import JSON

# Imports main tools:
from trulens_eval import Feedback, Tru, LiteLLM, Huggingface, TruBasicApp
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [3]:
from google.cloud import aiplatform
from vertexai.preview.generative_models import GenerativeModel, Part
import os

In [4]:
aiplatform.init(
            project = "ultra-heading-407815",
            location= "us-central1",
        )

In [5]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'cloud_credentials.json'

In [6]:
model = GenerativeModel("gemini-pro-vision")

In [7]:
from trulens_eval.feedback import Groundedness

In [8]:
provider = LiteLLM(model_engine='chat-bison-32k', max_output_tokens=2048, temperature=0.0)
grounded = Groundedness(groundedness_provider=provider)

In [9]:
# LLM-based feedback functions
f_criminality = Feedback(
    provider.criminality_with_cot_reasons,
    name="Criminality",
    higher_is_better=False,
).on_output()

f_insensitivity = Feedback(
    provider.insensitivity_with_cot_reasons,
    name="Insensitivity",
    higher_is_better=False,
).on_output()

f_maliciousness = Feedback(
    provider.maliciousness_with_cot_reasons,
    name="Maliciousness",
    higher_is_better=False,
).on_output()

# Moderation feedback functions
f_hate = Feedback(
    provider.harmfulness_with_cot_reasons,
    name="Harmfulness",
    higher_is_better=False
).on_output()

f_controvertial = Feedback(
    provider.controversiality_with_cot_reasons,
    name="Coherence",
    higher_is_better=False,
).on_output()

harmless_feedbacks = [
    f_criminality,
    f_insensitivity,
    f_maliciousness,
    f_hate,
    f_controvertial,
]

f_coherence = Feedback(
    provider.coherence_with_cot_reasons,
    name="Coherence",
    higher_is_better=True,
).on_output()


f_currectness = Feedback(
    provider.correctness_with_cot_reasons,
    name="Currectness",
    higher_is_better=True,
).on_output()

f_helpful = Feedback(
    provider.helpfulness_with_cot_reasons,
    name="Helpfulness",
    higher_is_better=True
).on_output()

f_conciseness = Feedback(
    provider.conciseness_with_cot_reasons,
    name="Conciseness",
    higher_is_better=True,
).on_output()


performance_feedback = [
    f_coherence,
    f_currectness,
    f_helpful,
    f_conciseness,
]

✅ In Criminality, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Insensitivity, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Maliciousness, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Harmfulness, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Coherence, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Coherence, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Currectness, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Helpfulness, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Conciseness, input text will be set to __record__.main_output or `Select.RecordOutput` .


In [28]:
all_feadbacks = harmless_feedbacks+performance_feedback

In [29]:
import base64
image_path = "image.jpeg"
img = open(image_path, "rb").read()
img_bytes = Part.from_data(base64.b64decode(base64.encodebytes(img)), mime_type="image/jpeg")

In [30]:
response = model.generate_content(["explain this image", img_bytes])
response.text

' This is a photo of a puppy. The breed is an Entlebucher Mountain Dog.'

In [31]:
prompt = "explain this image"

def llm_standalone(prompt):
    return model.generate_content([
        prompt
    ]+ [img_bytes, img_bytes]).text

In [32]:
llm_standalone("explain this 2 images, tell me whether it is same")

' The two images are the same.'

In [33]:
tru_app_recorder = TruBasicApp(llm_standalone, app_id="Sentiment bot", feedbacks=all_feadbacks)

In [34]:
with tru_app_recorder as records:
    tru_app_recorder.app("Explain the image")

A new object of type <class 'trulens_eval.tru_basic_app.TruWrapperApp'> at 0x7f118c611590 is calling an instrumented method <function TruWrapperApp._call at 0x7f11b1266840>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f11ac4628d0) using this function.


In [35]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   URL: http://localhost:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [36]:
tru.stop_dashboard()